In [4]:
!pip install -U -r requirements.txt


  Using cached langsmith-0.4.1-py3-none-any.whl.metadata (15 kB)


In [ ]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
os.environ["HF_TOKEN"] = getpass.getpass("Enter your HF API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain API key: ")

In [6]:
from langchain_groq import ChatGroq

# Replace this line
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Updated model
    temperature=0,
    max_tokens=4000,
    timeout=None
)
llm

ModuleNotFoundError: No module named 'langchain_groq'

In [4]:
import csv
import json
import uuid
import cassio  # Using cassio for AstraDB connection
import pandas as pd

from langchain_huggingface import HuggingFaceEmbeddings

from langchain_groq import ChatGroq

# AstraDB connection details
ASTRA_DB_APPLICATION_TOKEN =  getpass.getpass("Enter your Astra DB Token: ")
ASTRA_DB_ID =  getpass.getpass("Enter your Astra DB ID: ")


# Initialize AstraDB connection with Cassio
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID
)

# Embeddings setup
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# ChatGroq LLM setup
# Replace this line
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Updated model
    temperature=0,
    max_tokens=4000,
    timeout=None
)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

Entering data in the Vector DB for RAG

In [7]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
import cassio

# Create Cassandra vector store
astra_vector_store = Cassandra(
    embedding=embeddings,
    session=None,
    table_name="Projekt",
    keyspace=None
)

def process_csv(file_path, context_column, response_column):
    df = pd.read_csv(file_path)
    docs = []
    for _, row in df.iterrows():
        content = f"Context: {row[context_column]}\nResponse: {row[response_column]}"
        metadata = {"source": file_path, "type": "csv"}
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

def process_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    docs = []
    for item in data:
        content = f"Question: {item['question']}\nAnswer: {item['answer']}\nTopic: {item['topic']}"
        metadata = {"source": file_path, "type": "json", "topic": item['topic']}
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

# Process and add documents
docs = []
docs.extend(process_csv('/content/amod.csv', 'Context', 'Response'))
docs.extend(process_json('/content/cleaned_chats.json'))

# Add documents to the vector store
astra_vector_store.add_documents(docs)
print(f"Inserted {len(docs)} documents.")

# Create a vector store index wrapper
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

print("Vector index created successfully.")

Inserted 5641 documents.
Vector index created successfully.


In [17]:

import datetime
import warnings
from IPython.display import display, HTML
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import logging


# Filter out specific warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger('cassandra.protocol').setLevel(logging.ERROR)


# Memory setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the Conversational Retrieval Chain (RAG setup)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=astra_vector_store.as_retriever(search_kwargs={"k": 3}),
    memory=memory
)

def format_message(role, content):
    """Formats the message with HTML for better display in Colab."""
    color = "#4a76a8" if role == "Bot" else "#6c757d"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"""

        {role} - {timestamp}
        {content}

    """

def chat_with_rag():
    print("Hi, how are you feeling today?, You can type 'exit' or 'thankyou' to end the conversation.")

    while True:
        user_input = input("\nYou: ")

        if user_input.lower() == 'exit' or 'thankyou':
            display(HTML(format_message("Bot", "Goodbye! Have a great day!")))
            break

        if not user_input.strip():
            display(HTML(format_message("Bot", "Please enter a valid question or command.")))
            continue

        try:
            result = qa_chain.invoke({"question": user_input})
            # display(HTML(format_message("You", user_input)))
            display(HTML(format_message("Bot", result['answer'])))
        except Exception as e:
            error_message = f"Oops! Something went wrong. Please try again.\nError: {str(e)}"
            display(HTML(format_message("Bot", error_message)))

if __name__ == "__main__":
    chat_with_rag()

Hi, how are you feeling today?, You can type 'exit' or 'thankyou' to end the conversation.

You: hi i am depressed 


In [21]:
import datetime
import warnings
from IPython.display import display, HTML
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import logging


def format_message(role, content):
    """Formats the message with HTML for better display in Colab."""
    color = "#4a76a8" if role == "Bot" else "#6c757d"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"""
    <div style="margin-bottom: 10px; padding: 10px; border-radius: 5px; background-color: {color}; color: white;">
        <strong>{role}</strong> - {timestamp}<br>
        {content}
    </div>
    """


def chat_with_rag():
    # Memory setup (moved inside the function to ensure it's defined)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create the Conversational Retrieval Chain (RAG setup) (moved inside the function)
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=astra_vector_store.as_retriever(search_kwargs={"k": 3}),
        memory=memory
    )

    print("Hi, how are you feeling today?, You can type 'exit' or 'thankyou' to end the conversation.")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['exit', 'thankyou']:
            display(HTML(format_message("Bot", "Goodbye! Have a great day!")))
            break

        if not user_input:
            display(HTML(format_message("Bot", "Please enter a valid question or command.")))
            continue

        try:
            result = qa_chain.invoke({"question": user_input})
            display(HTML(format_message("Bot", result['answer'])))
        except Exception as e:
            error_message = f"Oops! Something went wrong. Please try again.\nError: {str(e)}"
            display(HTML(format_message("Bot", error_message)))

if __name__ == "__main__":
    chat_with_rag()


Hi, how are you feeling today?, You can type 'exit' or 'thankyou' to end the conversation.


KeyboardInterrupt: Interrupted by user

In [8]:
# prompt: installl gradio

!pip install gradio


In [9]:
import datetime
import warnings
import logging
import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from IPython.display import display, HTML

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger('cassandra.protocol').setLevel(logging.ERROR)

# Memory setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the Conversational Retrieval Chain (RAG setup)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=astra_vector_store.as_retriever(search_kwargs={"k": 3}),
    memory=memory
)

def chatbot_response(user_input):
    """Handles chatbot response and returns formatted output."""
    if user_input.lower() in ['exit', 'thankyou']:
        return "Goodbye! Have a great day! 😊"

    if not user_input.strip():
        return "Please enter a valid question or command."

    try:
        result = qa_chain.invoke({"question": user_input})
        return result['answer']
    except Exception as e:
        return f"Oops! Something went wrong. Please try again.\nError: {str(e)}"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Mental Health Chatbot")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Type your message here...")

    def update_chat(message, history):
        response = chatbot_response(message)
        history.append((message, response))
        return "", history

    user_input.submit(update_chat, [user_input, chatbot], [user_input, chatbot])

# Launch the interactive UI
if __name__ == "__main__":
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c39150462a2f679ebc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
